In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

2024-11-04 22:25:05.754206: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-04 22:25:06.025724: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-04 22:25:06.262868: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-04 22:25:06.464122: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-04 22:25:06.518417: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-04 22:25:06.930646: I tensorflow/core/platform/cpu_feature_gu

In [3]:
(train_ds , train_labels),(test_ds , test_labels)=tfds.load(
    "tf_flowers" , split= ["train[:70%]" , "train[:30%]"] , batch_size=-1 , as_supervised=True
)

2024-11-04 22:25:15.390596: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3488249856 exceeds 10% of free system memory.
2024-11-04 22:25:57.256968: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1494964224 exceeds 10% of free system memory.


In [ ]:
train_ds=tf.image.resize(train_ds,(150,150))

2024-11-04 22:26:22.732680: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 693630000 exceeds 10% of free system memory.


In [ ]:
test_ds=tf.image.resize(test_ds , (150,150))

In [ ]:
train_labels=to_categorical(train_labels , num_classes=5)

In [ ]:
test_labels=to_categorical(test_labels , num_classes=5)

In [ ]:
train_ds.shape

In [ ]:
test_ds.shape

In [ ]:
train_labels.shape

In [ ]:
test_labels.shape

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
base_model=VGG16(weights="imagenet" , include_top=False , input_shape=train_ds[0].shape)

In [ ]:
base_model.summary()

In [ ]:
base_model.trainable=False

In [ ]:
train_ds=preprocess_input(train_ds)

In [ ]:
test_ds=preprocess_input(test_ds)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten , Dense

In [ ]:
Flatten_layer=Flatten()
Dense_layer_1=Dense(50 , activation='relu')
Dense_layer_2=Dense(20 , activation='relu')
Last_layer=Dense(5 , activation='softmax')

In [ ]:
model=Sequential([
    base_model , 
    Flatten_layer,
    Dense_layer_1,
    Dense_layer_2,
    Last_layer
    
])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
es=EarlyStopping(monitor='val_accuracy' , mode='max' , restore_best_weights=True , patience=5 )

In [ ]:
model.compile(optimizer='adam' , loss='categorical_crossentropy' , metrics=["accuracy"])

In [ ]:
H=model.fit(train_ds , train_labels , validation_split=0.2 , epochs=1 , batch_size=34 , callbacks=es)

In [ ]:
loss , accu=model.evaluate(test_ds , test_labels)
print(loss)
print(accu)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure()
plt.plot(H.history["accuracy"])
plt.title("Plot of accuracy vs epochs")
plt.xlabel("epochs")
plt.ylabel("accuracy")

plt.show()

In [ ]:
import numpy as np
import random


In [ ]:
class_names=['dandelions' , 'daisy' , 'tulips' , 'sunflowers' , 'roses']
def prediction(model , test_images , test_labels , turn):
    for i in range(0,turn):
            random_idx=random.randint(0,test_images.shape[0])
            random_img=test_images[random_idx]
            random_labels=test_labels[random_idx].numpy()
        
            plt.imshow(random_img/255.0)
            plt.axis('off')
            plt.show()
        
            img_batch=np.expand_dims(random_img , axis=0)
            prediction=model.predict(img_batch)
            prediction_probabilities=prediction[0]
        
            for i,class_name in enumerate(class_names):
                print(f"{class_name}:{prediction_probabilities[i]*100:.2f}")
        
            ans=np.argmax(prediction_probabilities)
            predicted_flower=class_names[ans]
        
            print("our model says it is:" , predicted_flower)


prediction(model , test_ds ,test_labels , 3)
        